<a href="https://colab.research.google.com/github/Huzzi-10/PiLENS-Real-Time-AI-Based-Suspicious-Activity-Detection-with-NightVision-on-Raspberry-Pi/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------
#  A. Mount Drive & install
# ----------------------------
from google.colab import drive
drive.mount('/content/drive')

# Install required packages (ultralytics includes YOLO11 support)
!pip install -q ultralytics==0.12.0  # if version issues, omit version to get latest
!pip install -q torch torchvision opencv-python

# ----------------------------
#  B. Imports & Settings
# ----------------------------
import os, cv2, time
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from ultralytics import YOLO

# ---- PATHS (change only if your Drive layout differs) ----
DRIVE_ROOT = "/content/drive/MyDrive"
DATASET_DIR = os.path.join(DRIVE_ROOT, "Dataset")            # must contain Normal/ and Suspicious/
FRAMES_DIR  = os.path.join(DRIVE_ROOT, "Processed_Frames")   # output frames (will be created)
SAVE_MODEL_PATH = os.path.join(DRIVE_ROOT, "action_model.pth")

# ---- CLASSES (must match folder names in Dataset/) ----
CLASSES = ["Normal", "Suspicious"]

# ---- YOLO11 small/nano model name ----
YOLO_MODEL_NAME = "yolo11n.pt"   # using YOLOv11-nano as you wanted "yolov11n"

# ---- device / hyperparams ----
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEQUENCE_LEN = 16     # number of frames per sample
IMG_SIZE = 112
BATCH_SIZE = 2        # low-ish to avoid OOM on Colab; increase if GPU memory allows
EPOCHS = 6

print("Device:", DEVICE)

# ----------------------------
#  C. Prepare transforms & load YOLO
# ----------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
])

# Download/Load YOLO11n model (ultralytics will fetch from the hub if not present)
yolo = YOLO(YOLO_MODEL_NAME)

# Quick check: ensure dataset folders exist
for cls in CLASSES:
    path = os.path.join(DATASET_DIR, cls)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Expected dataset folder not found: {path}")
print("Dataset folders found:", os.listdir(DATASET_DIR))

# ----------------------------
#  D. Extract person crops from videos -> save frames to Drive
# ----------------------------
os.makedirs(FRAMES_DIR, exist_ok=True)
for cls in CLASSES:
    out_cls_dir = os.path.join(FRAMES_DIR, cls)
    os.makedirs(out_cls_dir, exist_ok=True)

print("Starting frame extraction (this may take time depending on dataset size)...")

def extract_person_frames(video_path, out_folder, skip_every_n=1):
    """Extract crops for detected persons from one video and save JPGs to out_folder.
       skip_every_n: save every nth frame to reduce frame count (1 -> every frame).
    """
    cap = cv2.VideoCapture(video_path)
    idx = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # optional skipping to reduce size: if idx % skip_every_n != 0: idx += 1; continue
        # Run YOLO on frame
        try:
            results = yolo(frame, verbose=False)
        except Exception as e:
            # fallback: skip problematic frames
            idx += 1
            continue
        # results can be a list of Result objects; use first result
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                if cls_id == 0:  # person
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    # Safety clamp
                    h, w = frame.shape[:2]
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(w-1, x2), min(h-1, y2)
                    crop = frame[y1:y2, x1:x2]
                    if crop is None or crop.size == 0:
                        continue
                    fname = os.path.join(out_folder, f"{os.path.basename(video_path).split('.')[0]}_{idx}.jpg")
                    cv2.imwrite(fname, crop)
                    saved += 1
        idx += 1
    cap.release()
    return saved

# Walk dataset folders and extract frames
total_saved = 0
for cls in CLASSES:
    in_folder = os.path.join(DATASET_DIR, cls)
    out_folder = os.path.join(FRAMES_DIR, cls)
    os.makedirs(out_folder, exist_ok=True)
    files = sorted(os.listdir(in_folder))
    for f in files:
        if f.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
            vpath = os.path.join(in_folder, f)
            print("Processing", vpath)
            saved = extract_person_frames(vpath, out_folder, skip_every_n=1)  # set skip_every_n=2 to save every other frame
            print(f"Saved {saved} frames for {f}")
            total_saved += saved

print("Frame extraction finished. Total saved:", total_saved)

# ----------------------------
#  E. Sequence Dataset: group frames into SEQUENCE_LEN sequences
# ----------------------------
class SequenceDataset(Dataset):
    def __init__(self, frames_root, seq_len=SEQUENCE_LEN, transform=transform):
        self.samples = []
        self.labels = []
        self.seq_len = seq_len
        self.transform = transform

        for lbl, cname in enumerate(CLASSES):
            folder = os.path.join(frames_root, cname)
            if not os.path.exists(folder):
                continue
            files = sorted([f for f in os.listdir(folder) if f.lower().endswith((".jpg",".png",".jpeg"))])
            paths = [os.path.join(folder, f) for f in files]
            # make sequences in a sliding window or non-overlapping; here non-overlapping
            for i in range(0, max(0, len(paths) - seq_len + 1), seq_len):
                seq = paths[i:i+seq_len]
                if len(seq) == seq_len:
                    self.samples.append(seq)
                    self.labels.append(lbl)

        print(f"Built dataset with {len(self.samples)} sequences.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq_paths = self.samples[idx]
        frames = []
        for p in seq_paths:
            img = cv2.imread(p)
            if img is None:
                # pad with zeros if file missing
                img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.transform(img)
            frames.append(img.unsqueeze(0))  # (1,3,H,W)
        frames = torch.cat(frames, dim=0)  # (T,3,H,W)
        # return as (T,C,H,W) — model expects (B,T,C,H,W) after batching
        return frames, self.labels[idx]

# Build dataset + dataloader
dataset = SequenceDataset(FRAMES_DIR, seq_len=SEQUENCE_LEN)
if len(dataset) == 0:
    raise RuntimeError("No sequences were created. Check that frames were saved into Processed_Frames/<class>/")
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# ----------------------------
#  F. Model: ResNet18 (feature extractor) + LSTM
# ----------------------------
class CNN_LSTM(nn.Module):
    def __init__(self, feature_dim=512, hidden_dim=128, num_classes=len(CLASSES)):
        super().__init__()
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        resnet.fc = nn.Identity()
        self.cnn = resnet  # returns 512-d features per frame
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        # flatten time & batch to feed through CNN
        x = x.view(B*T, C, H, W)
        feats = self.cnn(x)               # (B*T, 512)
        feats = feats.view(B, T, -1)      # (B, T, 512)
        lstm_out, _ = self.lstm(feats)    # (B, T, hidden_dim)
        out = self.fc(lstm_out[:, -1, :]) # use last timestep
        return out

model = CNN_LSTM().to(DEVICE)
print("Model created. Parameters:", sum(p.numel() for p in model.parameters()))

# ----------------------------
#  G. Training loop (simple)
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("Starting training...")
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    t0 = time.time()
    for X, y in loader:
        # X: (B, T, C, H, W) currently (B, T, C, H, W) due to collate defaults? Our __getitem__ returns (T,3,H,W)
        # If loader gives X as (B, T, C, H, W) or (B, T, C, H, W) — ensure shape:
        # We want (B, T, C, H, W)
        if X.dim() == 4:
            # case: (B*T,C,H,W) unlikely, but guard:
            pass
        # If X shape is (B, T, C, H, W) good; if (B, T, C, H, W) as expected.
        X = X.to(DEVICE).float()
        # ensure shape is (B, T, C, H, W)
        if X.dim() == 4 and X.shape[1] == SEQUENCE_LEN:
            # (B, T, H, W) incorrect layout; but our Dataset returns (T,C,H,W) and DataLoader stacks to (B,T,C,H,W)
            pass
        y = torch.tensor(y).to(DEVICE)

        optimizer.zero_grad()
        outputs = model(X)   # (B, num_classes)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    dt = time.time() - t0
    print(f"Epoch {epoch+1}/{EPOCHS}  Loss: {epoch_loss:.4f}  Time: {dt:.1f}s")

# ----------------------------
#  H. Save trained model to Drive
# ----------------------------
torch.save(model.state_dict(), SAVE_MODEL_PATH)
print("Model saved to:", SAVE_MODEL_PATH)

# ----------------------------
#  I. Notes & Next Steps
# ----------------------------
print("DONE pipeline. You can now download action_model.pth and copy to your Raspberry Pi for inference.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Ignored the following yanked versions: 8.0.129, 8.0.174, 8.0.177, 8.1.21, 8.1.31, 8.2.7, 8.2.47
ERROR: Ignored the following versions that require a different python version: 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7,<=3.11; 8.0.27 Requires-Python >=3.7,<=3.11; 8.0.28 Requires-Python 

100%|██████████| 44.7M/44.7M [00:00<00:00, 134MB/s]


Model created. Parameters: 11505474
Starting training...


/tmp/ipython-input-1371033146.py:225: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(DEVICE)


Epoch 1/6  Loss: 281.9593  Time: 263.2s
Epoch 2/6  Loss: 91.9694  Time: 166.9s
Epoch 3/6  Loss: 60.3379  Time: 162.1s
Epoch 4/6  Loss: 55.3160  Time: 157.6s
Epoch 5/6  Loss: 27.5835  Time: 156.8s
Epoch 6/6  Loss: 48.7931  Time: 155.9s
Model saved to: /content/drive/MyDrive/action_model.pth
DONE pipeline. You can now download action_model.pth and copy to your Raspberry Pi for inference.


In [ ]:
# ============================================================
#   A. Connect Google Drive
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================================
#   B. Install required libraries
# ============================================================
!pip install -q torch torchvision opencv-python

# ============================================================
#   C. Imports
# ============================================================
import os, cv2, time
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

# ============================================================
#   D. Paths & Settings (CHANGE ONLY THESE 3 IF NEEDED)
# ============================================================
DRIVE_ROOT = "/content/drive/MyDrive"
FRAMES_DIR  = os.path.join(DRIVE_ROOT, "Processed_Frames")   # already created by your pipeline
SAVE_MODEL_PATH = os.path.join(DRIVE_ROOT, "action_model.pth")

CLASSES = ["Normal", "Suspicious"]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

SEQUENCE_LEN = 16
IMG_SIZE = 112
BATCH_SIZE = 4           # you can increase to 8 if GPU supports
EPOCHS = 8              # <-- tum jitne chaaho yahan change kar do

print("Device:", DEVICE)
print("Frames directory:", FRAMES_DIR)

# ============================================================
#   E. Transform settings
# ============================================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
])

# ============================================================
#   F. Dataset Class (Sequence Dataset)
# ============================================================
class SequenceDataset(Dataset):
    def __init__(self, frames_root, seq_len=16, transform=None):
        self.samples = []
        self.labels = []
        self.seq_len = seq_len
        self.transform = transform

        for lbl, cname in enumerate(CLASSES):
            folder = os.path.join(frames_root, cname)
            if not os.path.exists(folder):
                print("Missing:", folder)
                continue

            files = sorted([f for f in os.listdir(folder)
                            if f.lower().endswith((".jpg", ".png", ".jpeg"))])

            paths = [os.path.join(folder, f) for f in files]

            # Non-overlapping sequence windows
            for i in range(0, max(0, len(paths) - seq_len + 1), seq_len):
                seq = paths[i:i+seq_len]
                if len(seq) == seq_len:
                    self.samples.append(seq)
                    self.labels.append(lbl)

        print(f"Total sequences created: {len(self.samples)}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq_paths = self.samples[idx]
        frames = []

        for p in seq_paths:
            img = cv2.imread(p)
            if img is None:
                img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.transform(img)
            frames.append(img.unsqueeze(0))  # (1,3,H,W)

        frames = torch.cat(frames, dim=0)  # (T,3,H,W)
        return frames, self.labels[idx]

# Dataset + DataLoader
dataset = SequenceDataset(FRAMES_DIR, seq_len=SEQUENCE_LEN, transform=transform)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

# ============================================================
#   G. Model Definition (ResNet18 + LSTM)
# ============================================================
class CNN_LSTM(nn.Module):
    def __init__(self, feature_dim=512, hidden_dim=128, num_classes=len(CLASSES)):
        super().__init__()
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        resnet.fc = nn.Identity()    # remove final layer
        self.cnn = resnet

        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: (B,T,C,H,W)
        B, T, C, H, W = x.shape
        x = x.reshape(B*T, C, H, W)
        feats = self.cnn(x)               # (B*T,512)
        feats = feats.reshape(B, T, -1)   # (B,T,512)
        lstm_out, _ = self.lstm(feats)
        out = self.fc(lstm_out[:, -1, :]) # last frame output
        return out

model = CNN_LSTM().to(DEVICE)

print("Model parameters:", sum(p.numel() for p in model.parameters()))

# ============================================================
#   H. Training Setup
# ============================================================
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ============================================================
#   I. Training Loop
# ============================================================
print("\n💥 Starting Training...\n")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    t0 = time.time()

    for X, y in loader:
        X = X.to(DEVICE).float()
        y = torch.tensor(y).to(DEVICE)

        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {total_loss:.4f} | Time: {time.time() - t0:.1f}s")

# ============================================================
#   J. Save Model
# ============================================================
torch.save(model.state_dict(), SAVE_MODEL_PATH)
print("\n🎉 Model Saved Successfully at:", SAVE_MODEL_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cuda
Frames directory: /content/drive/MyDrive/Processed_Frames
Total sequences created: 1679
Model parameters: 11505474

💥 Starting Training...



/tmp/ipython-input-3790665116.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).to(DEVICE)


Epoch 1/8 | Loss: 107.2894 | Time: 262.1s
Epoch 2/8 | Loss: 50.9269 | Time: 154.4s
Epoch 3/8 | Loss: 25.6259 | Time: 154.0s
Epoch 4/8 | Loss: 16.0332 | Time: 153.4s
Epoch 5/8 | Loss: 19.0876 | Time: 153.4s
Epoch 6/8 | Loss: 19.8068 | Time: 153.6s
Epoch 7/8 | Loss: 9.0061 | Time: 152.7s
Epoch 8/8 | Loss: 8.2380 | Time: 153.4s

🎉 Model Saved Successfully at: /content/drive/MyDrive/action_model.pth
